# **Multi-Layer Perceptron (MLP) for MNIST Digit Recognition**

This notebook provides a complete implementation of a Multi-Layer Perceptron (MLP) for the MNIST handwritten digit classification problem. It focuses on demonstrating the fundamental mechanics of a neural network from scratch using `numpy`, without relying on high-level deep learning frameworks like PyTorch or TensorFlow for the MLP's core logic. The dataset loading, however, leverages `torchvision` for convenience in a Colab environment, and then converts the data to NumPy arrays for further processing.

The notebook covers:

1.  **Theoretical Background:** A concise overview of MLP architecture, activation functions (Sigmoid, Softmax), Categorical Cross-Entropy (CCE) loss, regularization (L1, L2), forward pass, and backpropagation.
2.  **MNIST Data Handling:** Functions for loading and visualizing the MNIST dataset.
3.  **MLP Core Functions:** Detailed NumPy implementations of sigmoid, softmax, weight initialization, CCE loss calculation, forward propagation, backpropagation with gradient computations, and prediction.
4.  **MLP Training Loop:** The main training function incorporating mini-batch gradient descent with momentum and an adaptive learning rate.
5.  **Training and Evaluation:** Script to train the MLP on MNIST, evaluate its accuracy on training and test sets, and visualize the training loss.
6.  **Performance Visualization:** Displaying confusion matrices and sample predictions to assess model performance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Imported for consistency, though not directly used in 2D MNIST visualization.

np.random.seed(42)  # Set random seed for reproducibility of initial weights and data shuffling.

# **1. Theoretical Background**

## **1.1 Multi-Layer Perceptron (MLP) Architecture**
An MLP is a class of feedforward artificial neural network. It consists of at least three layers of nodes: an input layer, a hidden layer, and an output layer. Each node (neuron) in a layer is connected to every node in the subsequent layer, forming a densely connected or fully connected network. Information flows in one direction, from input to output, without loops. For this classification task, a two-layer MLP (one hidden layer) is employed.

## **1.2 Activation Functions**
Activation functions introduce non-linearity into the network, enabling it to learn complex, non-linear relationships in the data. Without them, an MLP would simply behave as a linear model.

* **Sigmoid Activation (Hidden Layer):** The sigmoid function maps any real-valued number to a value between 0 and 1. It's often used in hidden layers due to its non-linear and differentiable properties.
    $$ \sigma(z) = \frac{1}{1 + e^{-z}} $$
* **Sigmoid Derivative:** The derivative of the sigmoid function is crucial for the backpropagation algorithm, as it determines the gradient for updating weights.
    $$ \sigma'(z) = \sigma(z)(1 - \sigma(z)) $$
* **Softmax Activation (Output Layer):** The softmax function is used in the output layer for multi-class classification problems. It converts a vector of arbitrary real values into a probability distribution, where the sum of probabilities for all classes equals 1. This makes the output directly interpretable as class probabilities.
    $$ \text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}} \quad \text{for a K-class problem} $$

## **1.3 Cost Function: Categorical Cross-Entropy (CCE)**
The cost (or loss) function quantifies the discrepancy between the network's predicted outputs and the true labels. The goal of training is to minimize this cost. For multi-class classification with softmax output and one-hot encoded true labels, Categorical Cross-Entropy (CCE) is the standard choice.

For a single training sample $p$ and across all $C$ output classes, the CCE loss is given by:
$$ L(y_{true}, \hat{y}) = -\sum_{c=1}^{C} y_{true,c} \, \log(\hat{y}_c) $$
where $y_{true,c}$ is 1 if the true class is $c$ and 0 otherwise (one-hot encoding), and $\hat{y}_c$ is the predicted probability for class $c$. The total cost for a batch is the sum of individual sample costs.

## **1.4 Regularization**
Regularization techniques are applied to prevent overfitting, a phenomenon where the model performs well on training data but poorly on unseen data. They add a penalty term to the cost function based on the magnitude of the weights.

* **L1 Regularization (Lasso):** Adds a penalty proportional to the absolute value of the weights. This encourages sparsity, potentially driving some weights to exactly zero.
    $$ R_{L1}(\mathbf{W}) = \frac{\lambda_1}{2} \sum_{j} |w_j| $$
* **L2 Regularization (Ridge):** Adds a penalty proportional to the square of the weights. This discourages large weights, promoting a more distributed and less complex model.
    $$ R_{L2}(\mathbf{W}) = \frac{\lambda_2}{2} \sum_{j} w_j^2 $$

## **1.5 Forward Pass**
The forward pass is the process of computing the network's output predictions ($\hat{Y}$) for a given input ($X$) by propagating the input through each layer.

1.  **Input Layer to Hidden Layer:**
    * Augment the input $X$ with a bias term to form $A_0$ (input activations). If $X$ is $(N_{batch}, N_{features})$, then $A_0$ is $(N_{batch}, 1 + N_{features})$.
    * Compute the pre-activation values for the hidden layer: $Z_1 = A_0 W_1^T$. $W_1$ is the weight matrix for the hidden layer, with shape $(N_{hidden}, 1 + N_{features})$.
    * Apply the sigmoid activation function to $Z_1$ to get $A_1$ (hidden layer activations).
    * Augment $A_1$ with a bias term to form $A_1^{ext}$. $A_1^{ext}$ has shape $(N_{batch}, 1 + N_{hidden})$.

2.  **Hidden Layer to Output Layer:**
    * Compute the pre-activation values for the output layer: $Z_2 = A_1^{ext} W_2^T$. $W_2$ is the weight matrix for the output layer, with shape $(N_{output}, 1 + N_{hidden})$.
    * Apply the softmax activation function to $Z_2$ to get $\hat{Y}$ (the final predicted probabilities).

## **1.6 Backpropagation**
Backpropagation is the algorithm used to efficiently compute the gradients of the cost function with respect to each weight in the network. These gradients are then used to update the weights during training.

1.  **Output Layer Error ($\delta_2$):** For Categorical Cross-Entropy with Softmax output, the error signal at the output layer's pre-activation is remarkably simple:
    $$ \delta_2 = \hat{Y} - Y_{true} \quad \text{(where } \hat{Y} \text{ are softmax outputs and } Y_{true} \text{ are one-hot encoded labels)} $$
    If $\hat{Y}$ and $Y_{true}$ are $(N_{batch}, N_{output})$, then $\delta_2$ is also $(N_{batch}, N_{output})$.

2.  **Gradient for Output Weights ($W_2$):** The gradient for $W_2$ is computed as the outer product of the output error and the hidden layer activations (extended with bias).
    $$ \nabla_{W_2} J = \delta_2^T A_1^{ext} $$
    Resulting in a shape $(N_{output}, 1 + N_{hidden})$.

3.  **Hidden Layer Error ($\delta_1$):** This error signal is propagated back from the output layer to the hidden layer's pre-activations. It involves multiplying the output error by the output weights and then element-wise multiplying by the derivative of the hidden layer's activation function.
    $$ \delta_1 = (\delta_2 W_2)_{\text{excluding bias}} \odot \sigma'(Z_1) $$
    The term $(\delta_2 W_2)_{\text{excluding bias}}$ selects only the error signals relevant to the activated neurons in the hidden layer (excluding the bias connection). Resulting in a shape $(N_{batch}, N_{hidden})$.

4.  **Gradient for Hidden Weights ($W_1$):** The gradient for $W_1$ is computed as the outer product of the hidden layer error and the input activations (extended with bias).
    $$ \nabla_{W_1} J = \delta_1^T A_0 $$
    Resulting in a shape $(N_{hidden}, 1 + N_{features})$.

## **1.7 Gradient Descent Update Rule with Momentum**
Weights are updated iteratively to minimize the cost function. Gradient Descent moves weights in the direction opposite to the gradient. Momentum is added to accelerate convergence and dampen oscillations, by incorporating a fraction of the previous weight update.
$$ W \leftarrow W - \eta \nabla J + \alpha \Delta W_{prev} $$
where:
* $W$ is the current weight matrix.
* $\eta$ is the learning rate, controlling the step size.
* $\nabla J$ is the gradient of the cost function with respect to $W$.
* $\alpha$ is the momentum coefficient (typically between 0 and 1).
* $\Delta W_{prev}$ is the weight change from the previous iteration.

An adaptive learning rate (e.g., decaying over epochs) is often used to fine-tune convergence, allowing larger steps initially and smaller steps as training progresses to prevent overshooting the minimum.

# **2. MNIST Data Handling Functions**

This section provides functions for loading the MNIST dataset and visualizing its digits. The `mnist_load` function retrieves the image and label data, while `display_mnist` renders a grid of selected digits for visual inspection. These utilities are essential for preparing and understanding the dataset before training the MLP.

In [ ]:
def mnist_load(classes, Ntrain=None):
    """
    Loads MNIST data for specified classes using torchvision datasets, then converts to NumPy arrays.
    Optionally limits the number of training patterns.

    Args:
        classes (list): A list of integers representing the digits (0-9) to include.
        Ntrain (int, optional): Maximum number of training patterns to load. If None, loads all available.

    Returns:
        tuple: Xtrain (np.array) - Training images (N_train, 784),
               ytrain (np.array) - Training labels (N_train,),
               Xtest (np.array) - Test images (N_test, 784),
               ytest (np.array) - Test labels (N_test,).
    """
    import torch
    import torchvision
    from torchvision import datasets, transforms

    # Define a transformation to convert images to tensors and normalize them.
    # MNIST data is normalized with mean 0.1307 and std dev 0.3081 as per common practice.
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Load the training and test datasets. `download=True` will download if not present.
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    # Filter data based on specified classes and convert to NumPy arrays
    Xtrain_list, ytrain_list = [], []
    for img, label in train_dataset:
        if label in classes:
            Xtrain_list.append(img.numpy().flatten())  # Flatten 28x28 images to 784-dim vectors
            ytrain_list.append(label)

    Xtest_list, ytest_list = [], []
    for img, label in test_dataset:
        if label in classes:
            Xtest_list.append(img.numpy().flatten())
            ytest_list.append(label)

    Xtrain = np.array(Xtrain_list, dtype=np.float64) # Use float64 for numerical stability
    ytrain = np.array(ytrain_list, dtype=int)
    Xtest = np.array(Xtest_list, dtype=np.float64)
    ytest = np.array(ytest_list, dtype=int)

    # Shuffle training data and optionally limit its size
    shuffle_indices_train = np.random.permutation(len(ytrain))
    Xtrain = Xtrain[shuffle_indices_train]
    ytrain = ytrain[shuffle_indices_train]
    if Ntrain is not None and Ntrain < len(ytrain):
        Xtrain = Xtrain[:Ntrain]
        ytrain = ytrain[:Ntrain]

    # Shuffle test data
    shuffle_indices_test = np.random.permutation(len(ytest))
    Xtest = Xtest[shuffle_indices_test]
    ytest = ytest[shuffle_indices_test]

    return Xtrain, ytrain, Xtest, ytest

def display_mnist(X, example_width=None):
    """
    Displays 2D image data (e.g., MNIST digits) in a grid layout.
    The images are reshaped from flattened vectors back to 2D for display.

    Args:
        X (np.array): Input image data (N_images, N_pixels_flat).
        example_width (int, optional): Width of each square example image. If None, inferred.
    """
    if example_width is None:
        # Assume images are square, infer width from total pixels
        example_width = int(np.round(np.sqrt(X.shape[1])))

    plt.set_cmap('gray') # Set colormap to grayscale
    m, n = X.shape # m: number of images, n: number of flattened pixels
    example_height = n // example_width # Calculate height of each image

    # Compute number of rows and columns for the display grid
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))
    pad = 1 # Padding between images in the display grid

    # Create a blank array for the stitched display, filled with -1 (dark for normalized images)
    display_array = -np.ones(
        (pad + display_rows * (example_height + pad),
         pad + display_cols * (example_width + pad))
    )

    curr_ex = 0
    for j in range(display_rows):
        for i in range(display_cols):
            if curr_ex >= m:
                break # Stop if all examples are displayed
            
            # Get the maximum absolute value of the current image's pixels for normalization
            # This ensures consistent intensity mapping across images if data is not pre-normalized to 0-1
            max_val = np.max(np.abs(X[curr_ex, :]))
            
            # Copy the current example image into its patch in the display_array
            # Reshape the 1D image vector back to 2D (height, width)
            # Normalize by max_val to scale pixel values for consistent display
            display_array[
                pad + j * (example_height + pad) : pad + j * (example_height + pad) + example_height,
                pad + i * (example_width + pad) : pad + i * (example_width + pad) + example_width
            ] = X[curr_ex, :].reshape(example_height, example_width) / max_val
            curr_ex += 1
        if curr_ex >= m:
            break

    plt.figure(figsize=(display_cols*example_width/28, display_rows*example_height/28)) # Adjust figure size dynamically
    plt.imshow(display_array)
    plt.axis('off') # Hide axes for cleaner image display
    plt.title('Sample of MNIST Digits')
    plt.show()


# **3. MLP Core Functions**

This section implements the fundamental mathematical operations that constitute the Multi-Layer Perceptron. Each function is a self-contained unit responsible for a specific aspect of the MLP's behavior, such as activation, weight management, loss computation, and gradient calculation via backpropagation.

In [ ]:
def MLP_sigmoid(z):
    """
    Computes the sigmoid activation function element-wise.
    The sigmoid function squashes any real-valued number into the range (0, 1).
    Mathematically: \( \sigma(z) = 1 / (1 + e^{-z}) \)
    Args:
        z (np.array): Input array (pre-activation values).
    Returns:
        np.array: Output of the sigmoid function, with the same shape as `z`.
    """
    return 1.0 / (1.0 + np.exp(-z))

def MLP_sigmoid_gradient(Z):
    """
    Computes the derivative of the sigmoid function with respect to its input Z.
    This derivative is utilized during the backpropagation process.
    Mathematically: \( \sigma'(z) = \sigma(z)(1 - \sigma(z)) \)
    Args:
        Z (np.array): Input array (pre-activation values), for which the sigmoid and its derivative are calculated.
    Returns:
        np.array: Derivative of the sigmoid function, with the same shape as `Z`.
    """
    sig = MLP_sigmoid(Z)
    return sig * (1 - sig)

def MLP_softmax(rZ):
    """
    Computes the softmax activation function. This function is typically used in the output layer
    of a multi-class classification neural network to convert raw scores (logits) into probabilities.
    It ensures that the output values are between 0 and 1 and sum to 1 across classes for each sample.
    Mathematically: \( \text{softmax}(z_i) = e^{z_i} / \sum_{j=1}^{K} e^{z_j} \)
    
    Args:
        rZ (np.array): Input array of pre-activation values (logits). Expected shape (batch_size, n_output_classes).
    Returns:
        np.array: Output probabilities (batch_size, n_output_classes), where each row sums to 1.
    """
    # Subtract max for numerical stability to prevent overflow with large exponents
    # Axis=1 means calculate max for each row (each sample in the batch)
    exp_rZ = np.exp(rZ - np.max(rZ, axis=1, keepdims=True))
    # Sum of exponents for normalization (sum across classes for each sample)
    total = np.sum(exp_rZ, axis=1, keepdims=True)
    rA = exp_rZ / total
    return rA

def MLP_initialize_weights(model):
    """
    Initializes the weight matrices for the MLP. Weights are initialized uniformly
    between -1 and 1. Bias terms are incorporated as the first column in each weight matrix.
    This initialization helps in breaking symmetry and allows the network to learn diverse features.
    Args:
        model (dict): A dictionary containing network architecture parameters:
                      'n_hidden' (int): Number of neurons in the hidden layer.
                      'n_features' (int): Number of input features.
                      'n_output' (int): Number of neurons in the output layer (number of classes).
    Returns:
        tuple: W1 (np.array) - Weight matrix for the hidden layer, shape (n_hidden, n_features + 1).
               W2 (np.array) - Weight matrix for the output layer, shape (n_output, n_hidden + 1).
    """
    n_hidden = model['n_hidden']
    n_features = model['n_features']
    n_output = model['n_output']
    
    # W1: Weights connecting input layer (n_features + 1 for bias) to hidden layer (n_hidden).
    # Dimensions are (n_hidden, n_features + 1) because W1 will be multiplied by A0 @ W1.T.
    # Initialized in range [-1, 1].
    W1 = 2 * np.random.rand(n_hidden, n_features + 1) - 1
    
    # W2: Weights connecting hidden layer (n_hidden + 1 for bias) to output layer (n_output).
    # Dimensions are (n_output, n_hidden + 1).
    W2 = 2 * np.random.rand(n_output, n_hidden + 1) - 1
    return W1, W2

def encode_labels(y, k):
    """
    Performs one-hot encoding on a vector of integer class labels.
    One-hot encoding converts categorical labels into a binary vector representation,
    which is suitable for training with categorical cross-entropy loss.
    Args:
        y (np.array): A 1D NumPy array of original class labels (e.g., [0, 1, 2, 1, ...]).
        k (int): The total number of unique classes (e.g., 10 for MNIST digits 0-9).
    Returns:
        np.array: A 2D NumPy array of one-hot encoded labels, with shape (N_samples, k).
                  Each row corresponds to a sample, and each column to a class.
    """
    # Create a zero array with shape (number of samples, number of classes)
    onehot = np.zeros((len(y), k), dtype=int)
    # For each sample, set the column corresponding to its class label to 1
    # np.arange(len(y)) creates indices for rows, y for columns
    onehot[np.arange(len(y)), y] = 1  # Assuming y are 0-indexed classes
    return onehot

def L1_reg(lambda_reg, W1, W2):
    """
    Computes the L1 regularization cost. This penalty term is added to the loss function
    and is proportional to the sum of the absolute values of the weights (excluding bias terms).
    It promotes sparsity in the weight matrices.
    Args:
        lambda_reg (float): The regularization strength (lambda).
        W1 (np.array): Weight matrix for the hidden layer.
        W2 (np.array): Weight matrix for the output layer.
    Returns:
        float: The calculated L1 regularization cost.
    """
    # Sum the absolute values of all weights, excluding the bias column (index 0)
    return (lambda_reg / 2) * (np.sum(np.abs(W1[:, 1:])) + np.sum(np.abs(W2[:, 1:])))

def L2_reg(lambda_reg, W1, W2):
    """
    Computes the L2 regularization cost. This penalty term is added to the loss function
    and is proportional to the sum of the squared values of the weights (excluding bias terms).
    It discourages large weights and helps prevent overfitting.
    Args:
        lambda_reg (float): The regularization strength (lambda).
        W1 (np.array): Weight matrix for the hidden layer.
        W2 (np.array): Weight matrix for the output layer.
    Returns:
        float: The calculated L2 regularization cost.
    """
    # Sum the squared values of all weights, excluding the bias column (index 0)
    return (lambda_reg / 2) * (np.sum(W1[:, 1:]**2) + np.sum(W2[:, 1:]**2))

def MLP_CCESM_forward(X, W1, W2):
    """
    Performs the forward pass through the MLP with sigmoid activation in the hidden layer
    and softmax activation in the output layer.

    Args:
        X (np.array): Input data for the current batch, shape (batch_size, n_features).
        W1 (np.array): Weight matrix for the hidden layer, shape (n_hidden, n_features + 1).
        W2 (np.array): Weight matrix for the output layer, shape (n_output, n_hidden + 1).

    Returns:
        tuple: rA2 (np.array): Output activations (probabilities) from the softmax layer,
                                 shape (batch_size, n_output).
               A1_ext (np.array): Hidden layer activations, including the bias term,
                                   shape (batch_size, n_hidden + 1).
               A0 (np.array): Input activations, including the bias term,
                               shape (batch_size, n_features + 1).
               rZ1 (np.array): Pre-activation values of the hidden layer,
                               shape (batch_size, n_hidden).
               rZ2 (np.array): Pre-activation values of the output layer,
                               shape (batch_size, n_output).
    """
    batch_size = X.shape[0]

    # A0: Input layer activations augmented with a bias term (column of ones).
    # This allows the bias to be treated as a weight connected to a constant input of 1.
    A0 = np.hstack((np.ones((batch_size, 1)), X)) # Shape: (batch_size, n_features + 1)
    
    # rZ1: Pre-activation values for the hidden layer.
    # Computed by multiplying input activations (A0) with the transpose of hidden layer weights (W1.T).
    rZ1 = A0 @ W1.T # Shape: (batch_size, n_hidden)
    # A1: Activated hidden layer values using the sigmoid function.
    A1 = MLP_sigmoid(rZ1) # Shape: (batch_size, n_hidden)
    # A1_ext: Hidden layer activations augmented with a bias term for the output layer.
    A1_ext = np.hstack((np.ones((batch_size, 1)), A1)) # Shape: (batch_size, n_hidden + 1)
    
    # rZ2: Pre-activation values for the output layer.
    # Computed by multiplying extended hidden activations (A1_ext) with the transpose of output layer weights (W2.T).
    rZ2 = A1_ext @ W2.T # Shape: (batch_size, n_output)
    # rA2: Final output layer activations (predictions) using the softmax function.
    # Softmax converts logits into a probability distribution over classes.
    rA2 = MLP_softmax(rZ2) # Shape: (batch_size, n_output)
    
    return rA2, A1_ext, A0, rZ1, rZ2

def get_CCE_cost(y_enc, y_pred, model, W1, W2):
    """
    Computes the Categorical Cross-Entropy (CCE) cost, including L1 and L2 regularization terms.
    This function measures the performance of a classification model whose output is a probability value
    between 0 and 1. It's suitable for multi-class classification when true labels are one-hot encoded.
    Args:
        y_enc (np.array): One-hot encoded true labels, shape (batch_size, n_output_classes).
        y_pred (np.array): Predicted probabilities from the softmax layer, shape (batch_size, n_output_classes).
        model (dict): Dictionary containing model parameters 'l1' (L1 regularization weight) and 'l2' (L2 regularization weight).
        W1 (np.array): Weight matrix for the hidden layer.
        W2 (np.array): Weight matrix for the output layer.
    Returns:
        float: The total cost, which is the sum of CCE loss and regularization terms.
    """
    # Clip predictions to a small positive value to prevent log(0) which results in NaN or infinite loss.
    # Clipping also prevents values from reaching 1.0 which would make log(1-p) problematic in BCE (not CCE here).
    y_pred = np.clip(y_pred, 1e-12, 1.0 - 1e-12)

    # Calculate the Categorical Cross-Entropy (CCE) loss.
    # The formula is -sum(y_true * log(y_pred)) over all classes and all samples in the batch.
    # np.sum() without specifying axis will sum all elements of the resulting matrix.
    cce_loss = -np.sum(y_enc * np.log(y_pred))

    # Add L1 and L2 regularization terms to the cost.
    l1 = model['l1']
    l2 = model['l2']
    L1_term = L1_reg(l1, W1, W2)
    L2_term = L2_reg(l2, W1, W2)
    
    cost = cce_loss + L1_term + L2_term
    return cost

def get_CCESM_gradient(rA2, A1_ext, A0, rZ1, Y_enc, W1, W2, l1, l2):
    """
    Computes the gradients for the weights of the MLP using the backpropagation algorithm.
    This specific implementation is for Categorical Cross-Entropy loss with Softmax output activation.
    Args:
        rA2 (np.array): Output activations (predicted probabilities) from the softmax layer,
                        shape (batch_size, n_output_classes).
        A1_ext (np.array): Hidden layer activations, including the bias term,
                           shape (batch_size, n_hidden + 1).
        A0 (np.array): Input activations, including the bias term,
                       shape (batch_size, n_features + 1).
        rZ1 (np.array): Pre-activation values of the hidden layer, shape (batch_size, n_hidden).
        Y_enc (np.array): One-hot encoded true labels, shape (batch_size, n_output_classes).
        W1 (np.array): Current weight matrix for the hidden layer.
        W2 (np.array): Current weight matrix for the output layer.
        l1 (float): L1 regularization strength.
        l2 (float): L2 regularization strength.
    Returns:
        tuple: delta_W1_unscaled (np.array): Gradient for W1 (excluding regularization).
               delta_W2_unscaled (np.array): Gradient for W2 (excluding regularization).
    """
    # Step 1: Compute dL_dZ2 (Error signal at the output layer's pre-activation).
    # For CCE loss with Softmax, this derivative simplifies to (predicted_probabilities - true_one_hot_labels).
    dL_dZ2 = rA2 - Y_enc # Shape: (batch_size, n_output)
    
    # Step 2: Compute dL_dW2 (Gradient for the output layer weights W2).
    # This is calculated as the dot product of the transpose of the output error (dL_dZ2.T)
    # and the activations of the previous layer (A1_ext).
    dL_dW2 = dL_dZ2.T @ A1_ext # Shape: (n_output, n_hidden + 1)

    # Step 3: Compute dL_dA1 (Error propagated back to the hidden layer's activations).
    # This is done by multiplying the output error (dL_dZ2) with the output weights (W2).
    dL_dA1 = dL_dZ2 @ W2 # Shape: (batch_size, n_hidden + 1)
    
    # Step 4: Compute dL_drZ1 (Error signal at the hidden layer's pre-activation).
    # This involves taking the portion of dL_dA1 corresponding to actual neurons (excluding bias),
    # and element-wise multiplying it by the derivative of the hidden layer's activation function (sigmoid_gradient(rZ1)).
    sigma_prime_of_rZ1 = MLP_sigmoid_gradient(rZ1) # Shape: (batch_size, n_hidden)
    dL_drZ1 = dL_dA1[:, 1:] * sigma_prime_of_rZ1 # Shape: (batch_size, n_hidden)
    
    # Step 5: Compute dL_dW1 (Gradient for the hidden layer weights W1).
    # This is calculated as the dot product of the transpose of the hidden error (dL_drZ1.T)
    # and the input activations (A0).
    dL_dW1 = dL_drZ1.T @ A0 # Shape: (n_hidden, n_features + 1)

    # Step 6: Add regularization terms to the gradients (for non-bias weights).
    # The regularization terms' derivatives are added to the computed gradients.
    # Note: Regularization is applied only to the actual weights (columns from index 1 onwards),
    # not to the bias terms (column at index 0).
    delta_W1_unscaled = dL_dW1.copy()
    delta_W2_unscaled = dL_dW2.copy()
    
    # L1 and L2 derivatives are added to the non-bias parts of the gradients
    delta_W1_unscaled[:, 1:] += (l1 * np.sign(W1[:, 1:]) + 2 * l2 * W1[:, 1:])
    delta_W2_unscaled[:, 1:] += (l1 * np.sign(W2[:, 1:]) + 2 * l2 * W2[:, 1:])

    return delta_W1_unscaled, delta_W2_unscaled

def MLP_CCESM_predict(X, W1, W2):
    """
    Predicts class labels for a given input dataset X using the trained MLP.
    The prediction is made by performing a forward pass and then selecting the class
    with the highest probability from the softmax output.
    Args:
        X (np.array): Input data for prediction, shape (N_samples, n_features).
        W1 (np.array): Trained weight matrix for the hidden layer.
        W2 (np.array): Trained weight matrix for the output layer.
    Returns:
        np.array: Predicted 0-indexed class labels, shape (N_samples,).
    """
    # Perform forward pass to get output probabilities from the softmax layer
    rA2, _, _, _, _ = MLP_CCESM_forward(X, W1, W2)
    # Select the class with the highest probability for each sample
    y_pred = np.argmax(rA2, axis=1) 
    return y_pred


# **4. MLP Training Loop**

This section defines the core training function for the MLP. The `MLP_CCESM_train` function orchestrates the iterative learning process, employing mini-batch Gradient Descent with momentum. It manages the adaptive learning rate, shuffles the training data for each epoch, processes data in mini-batches, computes loss and gradients, and updates the network's weights. The training progress is monitored by recording the cost history.

In [ ]:
def MLP_CCESM_train(X_train, y_train, model):
    """
    Trains the Multi-Layer Perceptron model for multi-class classification.
    It uses mini-batch Gradient Descent with momentum and an adaptive learning rate.
    Args:
        X_train (np.array): Training input features, shape (N_train_samples, n_features).
        y_train (np.array): True training labels, shape (N_train_samples,).
        model (dict): Dictionary containing model parameters:
                      'n_output' (int): Number of output classes.
                      'l1' (float): L1 regularization weight.
                      'l2' (float): L2 regularization weight.
                      'eta' (float): Initial learning rate.
                      'alpha' (float): Momentum coefficient.
                      'epochs' (int): Number of full passes over the training data.
                      'minibatches' (int): Number of mini-batches to divide the training data into per epoch.
                      'decrease_const' (float): Constant for learning rate decay.
    Returns:
        tuple: model (dict) - The updated model dictionary containing trained weights (W1, W2)
                               and the cost history ('cost_history').
               W1 (np.array) - The trained weight matrix for the hidden layer.
               W2 (np.array) - The trained weight matrix for the output layer.
    """
    # Initialize weight matrices using the specified model parameters.
    W1, W2 = MLP_initialize_weights(model)
    
    # Extract relevant model parameters for easier access within the training loop.
    l1 = model['l1']
    l2 = model['l2']
    eta = model['eta']
    alpha = model['alpha']
    epochs = model['epochs']
    n_output = model['n_output']
    minibatches = model['minibatches']
    decrease_const = model['decrease_const']

    # One-hot encode all training labels once at the beginning.
    # This converts class labels (e.g., 0, 1, ..., 9) into binary vectors (e.g., [1,0,0,...]).
    Y_enc = encode_labels(y_train, n_output) # Shape: (N_train_samples, n_output)

    # Initialize previous weight changes for the momentum term. Set to zeros initially.
    delta_W1_prev = np.zeros_like(W1)
    delta_W2_prev = np.zeros_like(W2)
    
    # Initialize list to store the cost (loss) values observed during training.
    # This helps in monitoring convergence and debugging.
    model['cost_history'] = []

    num_observations = X_train.shape[0]
    
    # --- Main Training Loop: Iterates over each epoch ---
    for e in range(1, epochs + 1):
        print(f'Epoch: {e}')
        
        # Implement adaptive learning rate decay.
        # The learning rate decreases with each epoch, allowing for larger steps initially
        # and finer adjustments as the model approaches convergence.
        current_eta = eta / (1 + decrease_const * e)
        
        # Shuffle the training data indices at the start of each epoch.
        # This ensures that mini-batches are randomly sampled, preventing cyclical patterns
        # in weight updates and promoting better generalization.
        shuffled_indices = np.random.permutation(num_observations)
        X_shuffled = X_train[shuffled_indices, :]
        Y_enc_shuffled = Y_enc[shuffled_indices, :]

        # Calculate the size of each mini-batch.
        batch_size_per_minibatch = num_observations // minibatches
        
        # --- Mini-batch Loop: Iterates over each mini-batch within the current epoch ---
        for m_idx in range(minibatches):
            # Determine the start and end indices for the current mini-batch.
            start_idx = m_idx * batch_size_per_minibatch
            end_idx = start_idx + batch_size_per_minibatch
            
            # Extract the mini-batch data and corresponding one-hot encoded labels.
            X_mini_batch = X_shuffled[start_idx:end_idx, :]
            Y_enc_mini_batch = Y_enc_shuffled[start_idx:end_idx, :]
            
            # Perform the forward pass to get predictions for the current mini-batch.
            rA2, A1_ext, A0, rZ1, rZ2 = MLP_CCESM_forward(X_mini_batch, W1, W2)
            
            # Compute the cost (loss) for the current mini-batch.
            cost = get_CCE_cost(Y_enc_mini_batch, rA2, model, W1, W2)
            model['cost_history'].append(cost)
            
            # Compute gradients for weights using backpropagation.
            delta_W1_unscaled, delta_W2_unscaled = get_CCESM_gradient(
                rA2, A1_ext, A0, rZ1, Y_enc_mini_batch, W1, W2, l1, l2
            )
            
            # Calculate the actual weight changes including the learning rate.
            delta_W1 = current_eta * delta_W1_unscaled
            delta_W2 = current_eta * delta_W2_unscaled
            
            # Update the weight matrices using the gradient descent rule with momentum.
            # Momentum helps to smooth out updates and accelerate convergence in relevant directions.
            W1 = W1 - (delta_W1 + (alpha * delta_W1_prev))
            W2 = W2 - (delta_W2 + (alpha * delta_W2_prev))
            
            # Store the current weight changes to be used as previous changes in the next iteration (for momentum).
            delta_W1_prev = delta_W1
            delta_W2_prev = delta_W2
            
    # Store the final trained weight matrices in the model dictionary.
    model['W1'] = W1
    model['W2'] = W2
    return model, W1, W2


# **5. Main Script: Training and Evaluation**

This section serves as the main execution block for the MLP. It initializes the dataset, sets up model parameters, initiates the training process, and then evaluates the trained model's performance. Key metrics such as training and test accuracy are calculated, and the evolution of the training loss is plotted to observe convergence. A confusion matrix is also generated to provide a detailed breakdown of classification performance per class.

In [ ]:
plt.close('all') # Close all existing matplotlib figures to ensure clean plots

# --- Parameters for MNIST Data Loading ---
classes = list(range(10)) # All 10 digits (0-9) for classification
Ntrain = 60000            # Total number of training patterns to load
Ntest  = 10000            # Total number of test patterns to load (handled by mnist_load)

# --- Load MNIST Data ---
print("Loading MNIST dataset...")
Xtrain, ytrain, Xtest, ytest = mnist_load(classes, Ntrain)
print(f"Training data loaded: Xtrain shape={Xtrain.shape}, ytrain shape={ytrain.shape}")
print(f"Test data loaded: Xtest shape={Xtest.shape}, ytest shape={ytest.shape}")

# --- Display a Sample of Training Images ---
print("\nDisplaying a sample of MNIST training digits:")
# Select 64 random images to display in an 8x8 grid
idx = np.random.permutation(Xtrain.shape[0])[:64]
display_mnist(Xtrain[idx, :])

# --- Model Parameters for MLP ---
model = {
    'n_output': len(classes),      # Number of output neurons (equal to number of classes, 10 for MNIST)
    'n_features': Xtrain.shape[1], # Number of input features (28*28 = 784 pixels per image)
    'n_hidden': 50,                # Number of neurons in the single hidden layer (can be tuned)
    'l1': 0,                       # L1 regularization weight (set to 0 for no L1 regularization)
    'l2': 0.1,                     # L2 regularization weight (a small value to prevent overfitting)
    'epochs': 50,                  # Number of full passes through the training dataset
    'eta': 0.001,                  # Initial learning rate (step size for weight updates)
    'alpha': 0.001,                # Momentum coefficient (helps accelerate GD in relevant directions)
    'decrease_const': 0.00001,     # Constant for adaptive learning rate decay (eta / (1 + decrease_const * epoch))
    'minibatches': 50,             # Number of mini-batches per epoch. `Xtrain.shape[0] // 50` means approx 1200 samples per batch.
}

# --- Train MLP ---
print("\nStarting MLP training on MNIST dataset...")
model_trained, W1_trained, W2_trained = MLP_CCESM_train(Xtrain, ytrain, model)
print("MLP training complete.")

# --- Make Predictions on Training and Test Sets ---
ytrain_pred = MLP_CCESM_predict(Xtrain, W1_trained, W2_trained)
ytest_pred  = MLP_CCESM_predict(Xtest,  W1_trained, W2_trained)

# --- Compute and Display Accuracy ---
acc_train = np.mean(ytrain == ytrain_pred) # Calculate average accuracy over training set
print(f'\nTraining accuracy: {acc_train * 100:.2f}%')

acc_test = np.mean(ytest == ytest_pred)   # Calculate average accuracy over test set
print(f'Test accuracy: {acc_test * 100:.2f}%')

# --- Plot Training Loss History ---
plt.figure(figsize=(10, 6))
plt.plot(model_trained['cost_history'], color='blue', linewidth=2)
plt.title('MLP Training Loss over Mini-batch Iterations')
plt.xlabel('Mini-batch Iteration')
plt.ylabel('Loss (Categorical Cross-Entropy)')
plt.grid(True)
plt.show()

# --- Compute and Display Confusion Matrix for Test Set ---
print("\nConfusion Matrix (Test Set):")
num_classes = model['n_output']
confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

# Populate the confusion matrix
for i in range(len(ytest)):
    true_label = ytest[i]
    predicted_label = ytest_pred[i]
    confusion_matrix[true_label, predicted_label] += 1

print(confusion_matrix)

# --- Visualize Confusion Matrix (Confusion Chart) ---
plt.figure(figsize=(8, 7))
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues) # Use a blue colormap
plt.title('Confusion Chart (Test Set)')
plt.colorbar() # Add a color bar to indicate values

# Set ticks and labels for axes
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, np.arange(num_classes))
plt.yticks(tick_marks, np.arange(num_classes))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Add text annotations to each cell of the confusion matrix
thresh = confusion_matrix.max() / 2. # Threshold for text color (white on dark, black on light)
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, format(confusion_matrix[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if confusion_matrix[i, j] > thresh else "black")
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()


# **6. Visualization of Model Performance on Test Samples**

For high-dimensional input data like MNIST digits (784 features), a 3D decision surface plot is not directly interpretable. Instead, this section visualizes the model's performance by displaying a selection of test images along with their true labels and the MLP's predicted labels. Misclassified digits are highlighted to provide insights into where the model struggles. This direct visual inspection of predictions on actual data samples offers a more practical understanding of the MLP's learned capabilities.

In [ ]:
print("\nVisualizing sample test predictions:")

# Select a random subset of test images to display
num_display_samples = 25 # Display 25 samples in a 5x5 grid
display_indices = np.random.choice(len(ytest), num_display_samples, replace=False)

fig_pred, axes_pred = plt.subplots(5, 5, figsize=(10, 10))
axes_pred = axes_pred.flatten()

for i, ax in enumerate(axes_pred):
    if i < num_display_samples:
        img_idx = display_indices[i]
        image = Xtest[img_idx, :].reshape(28, 28) # Reshape 1D vector to 2D image
        true_label = ytest[img_idx]
        predicted_label = ytest_pred[img_idx]

        ax.imshow(image, cmap='gray')
        ax.axis('off')

        title_color = 'green' if true_label == predicted_label else 'red'
        ax.set_title(f'True: {true_label}\nPred: {predicted_label}', color=title_color, fontsize=10)
    else:
        ax.axis('off') # Hide unused subplots if num_display_samples is less than 25

plt.suptitle('Test Sample Predictions (Green: Correct, Red: Incorrect)', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
plt.show()
